# Travelling Salesman Problem (TSP)
## Introduction
The objective of a TSP is to find the optimal route through a network which visits all the nodes.  This notebook is a worked example of using the Icepack TSP solver to find a solution to a given TSP problem.

## Requirements
* **Protobuf** (Google's Protocol Buffers library) (https://pypi.org/project/protobuf/) `pip install protobuf`
* **Requests** (HTTP Python library) `pip install requests`
* **Plotting** (ipyleaflet) [installation instructions here](https://github.com/jupyter-widgets/ipyleaflet)
* Icepack API token of a key that has the Travelling Salesman Problem Solver enabled

## Protocol Buffers (Protobuf)
Developed for internal use by Google, Protocol Buffers are a flexible, efficient, automated mechanism for serialising structured data into a binary stream.

The proto definition file (`.proto`) defines the messages and services which is then complied using the protocol buffer compiler (`protoc`) to create code that can then be used to read and write messages. This repo includes precompiled message streams that are compatible with `protobuf 3.6.0` and above. If you require an older version of protobuf on your sub-system, feel free to recompile the `problem_pb2.py` and `tsp_mcvfz472gty6_pb2.py` files by running your own protobuf compiler (`protoc`) against the schema files (`problem.proto` and `tsp-mcvfz472gty6.proto`) as follows:

`protoc --python_out=. problem.proto`

`protoc --python_out=. tsp-mcvfz472gty6.proto`

This generates `problem_pb2` and `tsp_mcvfz472gty6_pb2` with the version of protobuf you have installed which can then be used the serialise the api messages.

### Protobuf Reference Materials
* [Developer Guide](https://developers.google.com/protocol-buffers/docs/overview)
* [Python Basics](https://developers.google.com/protocol-buffers/docs/pythontutorial)
* [Python Reference](https://developers.google.com/protocol-buffers/docs/reference/python-generated)

## Icepack Client Portal
The Icepack Client Portal can accessed [here](https://portal.icepack.ai).  This site gives your team access to an API key from which personal tokens can be generated to be used to access Icepack services. The site also allows for the monitoring of requests made by your team.

### API Key
All teams will initially have a single API key and, depending on your service plan, additional keys may be created.

Each team member will need to create a personal token for each key to which they are assigned.  This token is NOT stored by Icepack and it is up to you to ensure you save the token somewhere safe.

Tokens can be rotated at any point.  This won't affect other team member's tokens which will remain valid.  Members with the role of Team Owner or Admin can force all team members to rotate their personal tokens while members assigned the User role will only be able to rotate their own token.  

The Team Owner and Admin members can also edit the name and description of team keys as well as assign and unassign members and enable or disable Icepack services.

## TSP Set Up
The Icepack TSP service only requires the locations to be visited.  Each location requires an ID, and a pair of coordinates. For earth-routing, x and y (as cartesian coordinates) map to the location's longitude and latitude.

> Please check the regions currently covered [here](https://docs.scrudu.io/about/regions/#readout). If your geocodes fall outside this region we will assume a zero distance between points (and an unreasonable result will follow).

In [ ]:
import pandas
df = pandas.read_csv('../sample_data/publist.csv')
print(df.head())


### Create the TSP model
In this next section we perform the following:
* load a api helper class to assist with requests/serialisation to the icepack api
* load a tsp solve request class
* Populate the tsp model with geocodes

The api-helper assumes there's a `config.json` file which has your key and the end-point details. If you receive an error saying you haven't configured an api-key, please sign up on [the portal](portal.icepack.ai) and grab a key on the free tier services - no credit card details required!

In [ ]:
exec(open('apiHelper.py').read()) # import some helper classes which we've written for you.

api = apiHelper(modelType="tsp-mcvfz472gty6")

sr = tsp_mcvfz472gty6_pb2.SolveRequest()
sr.solveType = 0

m = sr.model

# or add them individually 
for index, row in df.iterrows():
  l = tsp_mcvfz472gty6_pb2.Geocode()
  l.id = row['id']
  l.x = row['X']
  l.y = row['Y']
  m.points.append(l)  

print(m) # that's what the model looks like, not too shabby.



### Send the model the api
We can simply post the post the model to the api using the method on the helper class we loaded earlier.

In [ ]:
reqId = api.Post(sr)
print(reqId) # if you want to see what the guid looks like.

### Get the answer
The guid returned above by the api is a unique token for us to retrieve the solution to the model we sent the api. We can now get the response for that request id (again, using a helper method).
If you're interested to see what's happening in the helper class, check out the `apiHelper.py` class.

In [ ]:
sol = api.Get(reqId)

print(sol.tour) # which is the sequence of geocode-id's we should follow to do perform an optimal tour!


### Reading the solution
You'll notice that the start and end point at both `The Oval Bar Dublin`: this is no mistake. We need to end the tour where we started, so this is indeed correct!

The api-helper also printed all the messages received by the api. This important to look at for more complex models with larger amount of master data. Warning messages and errors will be logged here to guide you on where the model might not be making sense to the api.

### Visualising the response
The api is quite smart, and knows that you're interested in using the road-network to execute your tour so it attached all the data we need to plot the optimal tour. 

We can left-join the tour onto our original pandas dataframe (which will provide a sorted order to the sequence) if we want to plot straight line segments. Lets have a quick look at an edge:

In [ ]:
tour= pandas.DataFrame(list(sol.tour), columns=['tour'])
tour.head()

In [ ]:

tourpts = pandas.merge(tour,df,left_on='tour',right_on='id', how = 'left').drop(['id'], axis=1)
tourpts.head()


### Edges
Each edge provides a starting location, an end location, and the long/lat series which can be interpreted as a line-string.

In [ ]:
print(sol.edges[0]) # just looking at the data contained in the first "edge" between stop 0 and stop 1

In [ ]:
# so we want a list of list of pairs (y,x => lat, lon).
from ipyleaflet import Map, Polyline, Circle, LayerGroup

#stops
locs = list()
for index, p in tourpts.iterrows():
    circle = Circle()
    circle.location = (p['Y'], p['X'])
    circle.radius = 10
    circle.color = "green"
    locs.append(circle)

#routes
routes = list()
for i, e in enumerate(sol.edges):
    route = list()
    for j, p in enumerate(e.geometry):
        route.append([p.y, p.x])  #ipyleaflet neets this in lat-lon format - cartographically correct. Cartesianially incorrect :-) 
    routes.append(route)
    
center = [df['Y'][1],df['X'][1]] # just use the first point as the cetner.
m = Map(scroll_wheel_zoom=True, center=center, zoom=12)
m.add_layer(Polyline(locations = routes, 
                     color="blue", 
                     fill = False))
m.add_layer(LayerGroup(layers=(locs)))
m


## Request Statistics
The statistics for all recent requests, including the solve duration, can be viewed in the dashboard of the [Icepack Client Portal](https://portal.icepack.ai/dashboard).